In [1]:
import os
while os.path.basename(os.getcwd()) != 'ecological-restoration':
    os.chdir('..')
    print(os.getcwd())

e:\bk\sarth\Github\ecological-restoration\Code\01_Species_Reintroductions
e:\bk\sarth\Github\ecological-restoration\Code
e:\bk\sarth\Github\ecological-restoration


In [8]:
import numpy as np
import pandas as pd

from numbalsoda import lsoda_sig, lsoda, dop853
from numba import njit, cfunc
import numba as nb

In [3]:
import gzip
import pickle
import pickletools

def PickleObj(obj, filepath):
    with gzip.open(filepath, "wb") as f:
        pickled = pickle.dumps(obj)
        optimized_pickle = pickletools.optimize(pickled)
        f.write(optimized_pickle)

def UnpickleObj(filepath):
    with gzip.open(filepath, 'rb') as f:
        p = pickle.Unpickler(f)
        obj = p.load()
    return obj

In [4]:
M_Real = UnpickleObj(
    os.path.join(
        os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', 'M_Real.pkl'
    )
)

M0 = M_Real['M_PL_041']


In [13]:
def make_rhs_func(M0):
    n, m = M0.shape
    D = np.concatenate((np.sum(np.array(M0), axis = 1),np.sum(np.array(M0), axis = 0)), axis = None)
    M0 = np.array(M0)

    alpha = 0.3
    h = 0.7
    mu = 1e-4
    gamma0 = 1.0
    p = 0.5
    bii = 1
    #bij = 0.01
    bij = 0

    @cfunc(lsoda_sig)
    def rhs(t, u, du, p):
        u_ = nb.carray(u, (n + m,))
        du_ = nb.carray(du, (n + m,))

        for i in range(n):
            term1 = mu + u_[i]*alpha 

            term2 = 0
            term3 = 0
            for j in range(n):
                if i != j:
                    term2 += bij*u_[j]
                elif i == j:
                    term2 += bii*u_[j]
            term2 = (-1.0)*term2*u_[i]
            for k in range(m):
                # if M0[i,k]:
                term3 += ((gamma0 / ((D[i])**p))*u_[n+k])*M0[i,k]
            term3 = (term3 / (1 + h*term3))*u_[i]

            du_[i] = term1 + term2 + term3
        
        for i in range(m):
            term1 = mu + u_[n+i]*alpha

            term2 = 0
            term3 = 0
            for j in range(m):
                if i != j:
                    term2 += bij*u_[n+j]
                elif i == j:
                    term2 += bii*u_[n+j]
            term2 = (-1.0)*term2*u_[n+i]
            for k in range(n):
                # if M0[k,i]:
                term3 += ((gamma0 / ((D[i+n])**p))*u_[k])*M0[k,i]
            term3 = (term3 / (1 + h*term3))*u_[i+n]

            du_[i+n] = term1 + term2 + term3
            
    return rhs

In [14]:
rhs = make_rhs_func(M0)
funcptr = rhs.address
t_eval = np.linspace(0, 100, 50)
n, m = M0.shape
u0 = np.ones(n)*10e-3
usol, success = lsoda(funcptr, u0, t_eval)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function pow>) found for signature:
 
 >>> pow(int64, float64*)
 
There are 4 candidate implementations:
[1m      - Of which 2 did not match due to:
      Overload of function 'pow': File: numba\core\typing\npydecl.py: Line 202.
        With argument(s): '(int64, float64*)':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Operator Overload in function 'pow': File: unknown: Line unknown.
        With argument(s): '(int64, float64*)':[0m
[1m       No match for registered cases:
        * (int64, int64) -> int64
        * (int64, uint64) -> int64
        * (uint64, int64) -> int64
        * (uint64, uint64) -> uint64
        * (float32, int32) -> float32
        * (float32, int64) -> float32
        * (float32, uint64) -> float32
        * (float64, int32) -> float64
        * (float64, int64) -> float64
        * (float64, uin

In [ ]:
rhs = make_pp_2d(M0)
funcptr = rhs.address
t_eval = np.linspace(0, 100, 50)
n, m = M0.shape
u0 = np.ones(n)*10e-3
usol, success = lsoda(funcptr, u0, t_eval, data = data)

In [3]:
@cfunc(lsoda_sig)
def rhs(t, u, du, p):
    du[0] = u[0]-u[0]*u[1]
    du[1] = u[0]*u[1]-u[1]*p[0]

funcptr = rhs.address # address to ODE function
u0 = np.array([5.,0.8]) # Initial conditions
data = np.array([1.0]) # data you want to pass to rhs (data == p in the rhs).
t_eval = np.linspace(0.0,50.0,1000) # times to evaluate solution

# integrate with lsoda method
usol, success = lsoda(funcptr, u0, t_eval, data = data)

# integrate with dop853 method
usol1, success1 = dop853(funcptr, u0, t_eval, data = data)

print(success, success1)

# usol = solution
# success = True/False

True True
